<a href="https://colab.research.google.com/github/harshraj22/DeepLearning/blob/main/submissions/HackerEarth-Snakes-In-The-Hood/hackerearths_snakes_in_the_hood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description

- Contest link [here](https://www.hackerearth.com/challenges/competitive/hackerearth-deep-learning-challenge-snake-breed-detection/machine-learning/identify-the-snake-breed-5-66d9a9f5/)

- Transfer Learning using resnet50 with 2 epochs gives score of 7.07897 on leaderboard

## Getting data from kaggle via api

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"harshraj22","key":"2d0e8c81a1bac612f94f75913ab7267d"}'}

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [1]:
import os

os.chdir('/content/drive/My Drive/Hackerearth')
os.getcwd()

'/content/drive/My Drive/Hackerearth'

In [ ]:
! kaggle datasets download -d hotsonhonet/hackerearths-snakes-in-the-hood

 87% 83.0M/95.9M [00:01<00:00, 71.6MB/s]
100% 95.9M/95.9M [00:01<00:00, 70.9MB/s]


In [ ]:
! ls && pwd

hackerearths-snakes-in-the-hood.zip
/content/drive/My Drive/Hackerearth


In [ ]:
! unzip hackerearths-snakes-in-the-hood.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/train/187f6f6636.jpg  
  inflating: dataset/train/18911a9924.jpg  
  inflating: dataset/train/189495070a.jpg  
  inflating: dataset/train/1894976a3c.jpg  
  inflating: dataset/train/1895f30c0a.jpg  
  inflating: dataset/train/18a6670c21.jpg  
  inflating: dataset/train/18a899f0ca.jpg  
  inflating: dataset/train/18ac42c83a.jpg  
  inflating: dataset/train/18b657edae.jpg  
  inflating: dataset/train/18ba3af0d9.jpg  
  inflating: dataset/train/18c835c0e8.jpg  
  inflating: dataset/train/18caacdaa8.jpg  
  inflating: dataset/train/18cc98b5bc.jpg  
  inflating: dataset/train/18e56d61b7.jpg  
  inflating: dataset/train/18edbcfba0.jpg  
  inflating: dataset/train/19029215c2.jpg  
  inflating: dataset/train/190317f717.jpg  
  inflating: dataset/train/190388a845.jpg  
  inflating: dataset/train/1905b12cca.jpg  
  inflating: dataset/train/190cdb9100.jpg  
  inflating: dataset/train/191533f0b1.jpg  
  inflating: dataset/trai

## Play around with data

In [2]:
import pandas as pd 
import numpy as np
import torch 
import torch.nn as nn

In [3]:
df = pd.read_csv('/content/drive/My Drive/Hackerearth/dataset/train.csv')
df.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


In [4]:
# number of unique breeds

len(df['breed'].unique())

35

In [5]:
labels = list(df['breed'].unique())

In [6]:
indexToLabel = labels
labelToIndex = dict((label, index) for index, label in enumerate(labels))

## Training process starts

In [7]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import cv2

#### create dataset

In [26]:
class ImgData(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.csv = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.csv)

  def __getitem__(self, idx):
    img = cv2.imread(self.root_dir + self.csv['image_id'][idx] + '.jpg')
    label = labelToIndex[self.csv['breed'][idx]]

    if self.transform:
      img = self.transform(img)

    return {'img': img, 'label': label, 'id': self.csv['image_id'][idx]}

In [9]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [10]:
data_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])

In [27]:
snakeDataset = ImgData('/content/drive/My Drive/Hackerearth/dataset/train.csv', '/content/drive/My Drive/Hackerearth/dataset/train/', data_transform)

In [71]:
snakeDataloader = DataLoader(snakeDataset, batch_size=32, num_workers=4)

#### create model
using transfer learning

In [13]:
from torchvision.models import resnet50
from torch.nn import functional as F
import torch.optim as optim

In [14]:
model = resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [15]:
for param in model.parameters():
  param.requires_grad = False

In [16]:
num_class = len(indexToLabel)

In [29]:
model.fc = nn.Sequential(
    nn.Linear(2048, 1000),
    nn.ReLU(inplace=True),
    nn.Linear(1000, 500),
    nn.ReLU(inplace=True),
    nn.Linear(500, num_class)
)

In [18]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda', index=0)

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

#### Training starts

In [73]:
from tqdm import tqdm

n_epoch = 25
model.train()

for epoch in range(n_epoch):
  epoch_loss = 0.0
  for data in tqdm(snakeDataloader, position=0, leave=False):  # for extra params, read https://stackoverflow.com/questions/41707229/tqdm-printing-to-newline
    img, label = data['img'], data['label']
    img = img.to(device)
    label = torch.tensor(label).to(device)

    outs = model(img)
    loss = criterion(outs, label)
    
    optimizer.zero_grad()

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item() * img.size()[0]

  print('\n epoch: ', epoch, ' loss: ', epoch_loss)


  0%|          | 0/173 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
  0%|          | 0/173 [00:00<?, ?it/s]


 epoch:  0  loss:  15357.488353729248


  0%|          | 0/173 [00:00<?, ?it/s]


 epoch:  1  loss:  15317.400820732117


  0%|          | 0/173 [00:00<?, ?it/s]

...

  0%|          | 0/173 [00:00<?, ?it/s]


 epoch:  22  loss:  14965.529074668884


  0%|          | 0/173 [00:00<?, ?it/s]


 epoch:  23  loss:  14987.206632614136



 epoch:  24  loss:  14913.412317752838


In [74]:
model_path = '/content/drive/My Drive/Hackerearth/model.t7'
torch.save(model.state_dict(), model_path)

## Prediction 

In [58]:
test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

In [59]:
snakeTestDataset = ImgData('/content/drive/My Drive/Hackerearth/dataset/test.csv', '/content/drive/My Drive/Hackerearth/dataset/test/', test_transform)

In [60]:
snakeTestDataloader = DataLoader(snakeTestDataset, batch_size=1) 

In [61]:
test_csv = pd.read_csv('/content/drive/My Drive/Hackerearth/dataset/test.csv')

In [62]:
preds = pd.DataFrame(columns=['image_id', 'breed'])

In [75]:
model.eval()

for index, data in enumerate(tqdm(snakeTestDataloader, position=0, leave=True)):
  img, label, id = data['img'], data['label'], data['id']
  img = img.to(device)
  outs = model(img)
  cls = torch.argmax(outs, dim=1).item()

  name = id
  label = indexToLabel[cls]
  preds = preds.append({'image_id': name, 'breed': label}, ignore_index=True)

100%|██████████| 2361/2361 [01:02<00:00, 37.55it/s]


In [76]:
preds['image_id'] = [id[0] for id in preds['image_id']]

In [77]:
preds.head()

,image_id,breed
0,7,thamnophis-sirtalis
1,3,lampropeltis-triangulum
2,d,lampropeltis-californiae
3,1,pantherophis-obsoletus
4,8,storeria-dekayi


In [78]:
set(preds['image_id']) == set(test_csv['image_id']) 

False

In [82]:
preds.to_csv('preds.csv', index=False) 

In [83]:
len(preds), len(test_csv)

(2361, 2361)

In [81]:
preds = preds.tail(2361)

In [84]:
''' details
epoch: 12
score on leaderboard: 15.87
model weights saved: True
'''

' details\nepoch: 12\nscore on leaderboard: 15.87\nmodel weights saved: True\n'